<a href="https://colab.research.google.com/github/jinchen1036/Product-Price-Prediction/blob/main/experiment/AttributeEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import itertools
import numpy as np
import pandas as pd

In [2]:
# read sample data
sample_df = pd.read_csv("https://raw.githubusercontent.com/jinchen1036/Product-Price-Prediction/main/data/sample_data.csv")
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   train_id           1000 non-null   int64  
 1   name               1000 non-null   object 
 2   item_condition_id  1000 non-null   int64  
 3   category_name      996 non-null    object 
 4   brand_name         561 non-null    object 
 5   price              1000 non-null   float64
 6   shipping           1000 non-null   int64  
 7   item_description   1000 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 62.6+ KB


In [3]:
# drop row with missing value  as it not need in this task
sample_df.dropna(inplace=True)
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558 entries, 6 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   train_id           558 non-null    int64  
 1   name               558 non-null    object 
 2   item_condition_id  558 non-null    int64  
 3   category_name      558 non-null    object 
 4   brand_name         558 non-null    object 
 5   price              558 non-null    float64
 6   shipping           558 non-null    int64  
 7   item_description   558 non-null    object 
dtypes: float64(1), int64(3), object(4)
memory usage: 39.2+ KB


In [ ]:
sample_df.head(5)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
6,1477709,Victoria's Secret Alabama sleep pants,2,Women/Other/Other,Victoria's Secret,8.0,1,University of Alabama pajama pants. Size XS. S...
8,711453,Black top,3,Women/Tops & Blouses/Blouse,Mossimo,8.0,1,Black off the shoulder top. Size small
10,834405,Express flip flops,3,Women/Shoes/Sandals,Express,9.0,0,No description yet
13,85037,Sprint Htc one,2,Electronics/Cell Phones & Accessories/Cell Pho...,HTC,43.0,0,fresh format ready to go with otterbox low pri...
14,44256,Lularoe XS Irma Mickey Lularoe Logo Pink,1,Women/Tops & Blouses/Tunic,Disney Collection,42.0,0,Brand New


## Number of Categories for `brand_name` and `category_name`

In [ ]:
print("Number of categories for category_name attribute is ",len(sample_df.category_name.unique()))

Number of categories for category_name attribute is  183


In [ ]:
for category in sample_df.category_name.unique()[:5]:
  print(category)

Women/Other/Other
Women/Tops & Blouses/Blouse
Women/Shoes/Sandals
Electronics/Cell Phones & Accessories/Cell Phones & Smartphones
Women/Tops & Blouses/Tunic


In [ ]:
print("Number of categories for brand_name attribute is ",len(sample_df.brand_name.unique()))

Number of categories for brand_name attribute is  233


In [ ]:
for brand_name in sample_df.brand_name.unique()[:5]:
  print(brand_name)

Victoria's Secret
Mossimo
Express
HTC
Disney Collection


## One Hot Encoding - brand_name

In [29]:
 # using get_dummies, drop one column as it is correlated with other column
 encode_df = pd.get_dummies(sample_df[['brand_name']], drop_first=True)
 encode_df.head(5)

,brand_name_Abercrombie & Fitch,brand_name_Adidas,brand_name_Aeropostale,brand_name_Air Jordan,brand_name_Alexander Wang,brand_name_Alice + Olivia,brand_name_American Apparel,brand_name_American Eagle,brand_name_American Fighter,brand_name_American Girl ®,brand_name_Angie,brand_name_Ann Taylor LOFT,brand_name_Anthropologie,brand_name_Apple,brand_name_Avent,brand_name_BISOU BISOU,brand_name_BKE,brand_name_Baby Einstein,brand_name_Balmain,brand_name_Barbie,brand_name_Bass,brand_name_Bath & Body Works,brand_name_Beats by Dr. Dre,brand_name_Belly Bandit®,brand_name_Ben Nye,brand_name_Benefit,brand_name_Benetton,brand_name_Betsey Johnson,brand_name_Billabong,brand_name_Brandy Melville,brand_name_Buckle,brand_name_Calvin Klein,brand_name_Cardinal Industries,brand_name_Carhartt,brand_name_Carlos Santana,brand_name_Carter's,brand_name_Cato,brand_name_Chaco,brand_name_Champion,brand_name_Chanel,...,brand_name_Stussy,brand_name_Supreme,brand_name_Swarovski,brand_name_TOMS,brand_name_Target,brand_name_Tarte,brand_name_The Limited,brand_name_The North Face,brand_name_Thrasher Magazine,brand_name_Too Faced,brand_name_Tory Burch,brand_name_Toysmith,brand_name_Triangl,brand_name_True Religion Brand Jeans,brand_name_UGG Australia,brand_name_Under Armour,brand_name_Urban Decay,brand_name_VANS,brand_name_Venus,brand_name_Vera Bradley,brand_name_Versace,brand_name_Victoria's Secret,brand_name_Vintage,brand_name_WWE,brand_name_Wet Seal,brand_name_Wet n Wild,brand_name_Whisker City,brand_name_White House Black Market,brand_name_Xbox,brand_name_Xhilaration,brand_name_YMI,brand_name_Yves Saint Laurent,brand_name_ZARA,brand_name_Zella,brand_name_beautyblender®,brand_name_kate spade new york,brand_name_lululemon athletica,brand_name_rue,brand_name_theBalm,brand_name_vineyard vines
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
merge_df = pd.concat([sample_df, encode_df], axis=1)
merge_df.head(5)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,brand_name_Abercrombie & Fitch,brand_name_Adidas,brand_name_Aeropostale,brand_name_Air Jordan,brand_name_Alexander Wang,brand_name_Alice + Olivia,brand_name_American Apparel,brand_name_American Eagle,brand_name_American Fighter,brand_name_American Girl ®,brand_name_Angie,brand_name_Ann Taylor LOFT,brand_name_Anthropologie,brand_name_Apple,brand_name_Avent,brand_name_BISOU BISOU,brand_name_BKE,brand_name_Baby Einstein,brand_name_Balmain,brand_name_Barbie,brand_name_Bass,brand_name_Bath & Body Works,brand_name_Beats by Dr. Dre,brand_name_Belly Bandit®,brand_name_Ben Nye,brand_name_Benefit,brand_name_Benetton,brand_name_Betsey Johnson,brand_name_Billabong,brand_name_Brandy Melville,brand_name_Buckle,brand_name_Calvin Klein,...,brand_name_Stussy,brand_name_Supreme,brand_name_Swarovski,brand_name_TOMS,brand_name_Target,brand_name_Tarte,brand_name_The Limited,brand_name_The North Face,brand_name_Thrasher Magazine,brand_name_Too Faced,brand_name_Tory Burch,brand_name_Toysmith,brand_name_Triangl,brand_name_True Religion Brand Jeans,brand_name_UGG Australia,brand_name_Under Armour,brand_name_Urban Decay,brand_name_VANS,brand_name_Venus,brand_name_Vera Bradley,brand_name_Versace,brand_name_Victoria's Secret,brand_name_Vintage,brand_name_WWE,brand_name_Wet Seal,brand_name_Wet n Wild,brand_name_Whisker City,brand_name_White House Black Market,brand_name_Xbox,brand_name_Xhilaration,brand_name_YMI,brand_name_Yves Saint Laurent,brand_name_ZARA,brand_name_Zella,brand_name_beautyblender®,brand_name_kate spade new york,brand_name_lululemon athletica,brand_name_rue,brand_name_theBalm,brand_name_vineyard vines
6,1477709,Victoria's Secret Alabama sleep pants,2,Women/Other/Other,Victoria's Secret,8.0,1,University of Alabama pajama pants. Size XS. S...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,711453,Black top,3,Women/Tops & Blouses/Blouse,Mossimo,8.0,1,Black off the shoulder top. Size small,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,834405,Express flip flops,3,Women/Shoes/Sandals,Express,9.0,0,No description yet,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,85037,Sprint Htc one,2,Electronics/Cell Phones & Accessories/Cell Pho...,HTC,43.0,0,fresh format ready to go with otterbox low pri...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,44256,Lularoe XS Irma Mickey Lularoe Logo Pink,1,Women/Tops & Blouses/Tunic,Disney Collection,42.0,0,Brand New,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
# drop brand_name as it no longer need
merge_df.drop(['brand_name'], axis=1, inplace=True)
merge_df.head(3)

,train_id,name,item_condition_id,category_name,price,shipping,item_description,brand_name_Abercrombie & Fitch,brand_name_Adidas,brand_name_Aeropostale,brand_name_Air Jordan,brand_name_Alexander Wang,brand_name_Alice + Olivia,brand_name_American Apparel,brand_name_American Eagle,brand_name_American Fighter,brand_name_American Girl ®,brand_name_Angie,brand_name_Ann Taylor LOFT,brand_name_Anthropologie,brand_name_Apple,brand_name_Avent,brand_name_BISOU BISOU,brand_name_BKE,brand_name_Baby Einstein,brand_name_Balmain,brand_name_Barbie,brand_name_Bass,brand_name_Bath & Body Works,brand_name_Beats by Dr. Dre,brand_name_Belly Bandit®,brand_name_Ben Nye,brand_name_Benefit,brand_name_Benetton,brand_name_Betsey Johnson,brand_name_Billabong,brand_name_Brandy Melville,brand_name_Buckle,brand_name_Calvin Klein,brand_name_Cardinal Industries,...,brand_name_Stussy,brand_name_Supreme,brand_name_Swarovski,brand_name_TOMS,brand_name_Target,brand_name_Tarte,brand_name_The Limited,brand_name_The North Face,brand_name_Thrasher Magazine,brand_name_Too Faced,brand_name_Tory Burch,brand_name_Toysmith,brand_name_Triangl,brand_name_True Religion Brand Jeans,brand_name_UGG Australia,brand_name_Under Armour,brand_name_Urban Decay,brand_name_VANS,brand_name_Venus,brand_name_Vera Bradley,brand_name_Versace,brand_name_Victoria's Secret,brand_name_Vintage,brand_name_WWE,brand_name_Wet Seal,brand_name_Wet n Wild,brand_name_Whisker City,brand_name_White House Black Market,brand_name_Xbox,brand_name_Xhilaration,brand_name_YMI,brand_name_Yves Saint Laurent,brand_name_ZARA,brand_name_Zella,brand_name_beautyblender®,brand_name_kate spade new york,brand_name_lululemon athletica,brand_name_rue,brand_name_theBalm,brand_name_vineyard vines
6,1477709,Victoria's Secret Alabama sleep pants,2,Women/Other/Other,8.0,1,University of Alabama pajama pants. Size XS. S...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,711453,Black top,3,Women/Tops & Blouses/Blouse,8.0,1,Black off the shoulder top. Size small,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,834405,Express flip flops,3,Women/Shoes/Sandals,9.0,0,No description yet,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## One Hot Encoding - category_name

### Get all unqiue category names

In [6]:
category_name_group = list(sample_df.category_name.unique())

In [18]:
category_names = list(itertools.chain(*map(lambda x: x.split('/'), category_name_group)))
len(category_names)

551

In [19]:
category_names_unique = set(category_names)
len(category_names_unique)

209

# Encoding category_name

In [24]:
category_df = pd.DataFrame(0, index=np.arange(len(sample_df)), columns=list(category_names_unique))
category_df.head(3)

,Full-Length,Animation,Athletic Apparel,Swim Trunks,Fitness accessories,Motorcycle,Mules & Clogs,Housewares,Sandals,Kids' Bedding,Boys 2T-5T,Boys 0-24 Mos,Tops & Blouses,Video Games & Consoles,Cleansers,Bathing Accessories,Sets,eBook Readers,Bedding,NFL,Cameras & Photography,Exercise,Cowl Neck,Sunglasses,Hats,"Sweatshirt, Pullover","TV, Audio & Surveillance",Clocks,Athletic,Bowl,Electronics,Shoes,Games,Home Décor Accents,Toy Remote Control & Play Vehicles,Kids,Skin Care,"Cases, Covers & Skins",Bottle-Feeding,Women's Accessories,...,Scarves & Wraps,Arts & Crafts,Personal Care,Men's Accessories,Earrings,G-Strings & Thongs,Backpacks & Carriers,Bracelets,Fashion Sneakers,Hoodie,Duvet Covers & Sets,Brushes & Applicators,Office supplies,Men,Jeans,T-shirts,Fragrance,Flats,Tools & Accessories,Tracksuits & Sweats,Top & T-shirts,Blouse,Jean Jacket,Video Game,Others,Action Figures & Statues,Building Toys,Home Fragrance,Clothing,Bras,"Tank, Cami",Pumps,Toys,"Pants, Tights, Leggings",Lips,Cell Phones & Accessories,Collectibles,Pants,Handbag,Yoga & Pilates
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
sample_df["category_name_group"] = sample_df.category_name.apply(lambda x: x.split('/'))

In [26]:
sample_df.head(3)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,category_name_group
6,1477709,Victoria's Secret Alabama sleep pants,2,Women/Other/Other,Victoria's Secret,8.0,1,University of Alabama pajama pants. Size XS. S...,"[Women, Other, Other]"
8,711453,Black top,3,Women/Tops & Blouses/Blouse,Mossimo,8.0,1,Black off the shoulder top. Size small,"[Women, Tops & Blouses, Blouse]"
10,834405,Express flip flops,3,Women/Shoes/Sandals,Express,9.0,0,No description yet,"[Women, Shoes, Sandals]"


In [27]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(sparse_output=True)

category_encode_df = pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(sample_df.pop('category_name_group')),
                index=sample_df.index,
                columns=mlb.classes_)
category_encode_df.head(3)

,"Above Knee, Mini",Accessories,Action Figure,Action Figures & Statues,Animation,Apparel,Arts & Crafts,Asymmetrical Hem,Athletic,Athletic Apparel,Baby & Toddler Toys,Backpacks & Carriers,"Backpacks, Bags & Briefcases",Bags and Purses,Bath,Bath & Body,Bathing Accessories,Beauty,Bedding,Belts,Blazer,Blouse,Body,Boot Cut,Boots,Bottle-Feeding,Bottoms,Bowl,Boyfriend,Boys (4+),Boys 0-24 Mos,Boys 2T-5T,Bracelets,Bras,Brushes & Applicators,Building Toys,Button Down Shirt,Button-Front,Cables & Adapters,Cameras & Photography,...,Suits & Blazers,Sunglasses,Sweaters,Sweats & Hoodies,"Sweatshirt, Pullover",Swim Trunks,Swimwear,T-Shirts,T-shirts,"TV, Audio & Surveillance",Tablet,"Tank, Cami",Tools & Accessories,Top & T-shirts,Tops,Tops & Blouses,Tops & T-Shirts,Totes & Shoppers,Toy,Toy Remote Control & Play Vehicles,Toys,Tracksuits & Sweats,Trading Cards,Tunic,Two-Piece,Underwear,Vases,Video Game,Video Games & Consoles,Vintage & Collectibles,Wallets,Watch,Watches,Women,Women's Accessories,Women's Handbags,Writing,Yoga & Pilates,eBook Readers,iPad
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [32]:
merge_df = merge_df.join(category_encode_df)

In [34]:
merge_df.drop(['category_name'], axis=1, inplace=True)
merge_df.head(3)

,train_id,name,item_condition_id,price,shipping,item_description,brand_name_Abercrombie & Fitch,brand_name_Adidas,brand_name_Aeropostale,brand_name_Air Jordan,brand_name_Alexander Wang,brand_name_Alice + Olivia,brand_name_American Apparel,brand_name_American Eagle,brand_name_American Fighter,brand_name_American Girl ®,brand_name_Angie,brand_name_Ann Taylor LOFT,brand_name_Anthropologie,brand_name_Apple,brand_name_Avent,brand_name_BISOU BISOU,brand_name_BKE,brand_name_Baby Einstein,brand_name_Balmain,brand_name_Barbie,brand_name_Bass,brand_name_Bath & Body Works,brand_name_Beats by Dr. Dre,brand_name_Belly Bandit®,brand_name_Ben Nye,brand_name_Benefit,brand_name_Benetton,brand_name_Betsey Johnson,brand_name_Billabong,brand_name_Brandy Melville,brand_name_Buckle,brand_name_Calvin Klein,brand_name_Cardinal Industries,brand_name_Carhartt,...,Suits & Blazers,Sunglasses,Sweaters,Sweats & Hoodies,"Sweatshirt, Pullover",Swim Trunks,Swimwear,T-Shirts,T-shirts,"TV, Audio & Surveillance",Tablet,"Tank, Cami",Tools & Accessories,Top & T-shirts,Tops,Tops & Blouses,Tops & T-Shirts,Totes & Shoppers,Toy,Toy Remote Control & Play Vehicles,Toys,Tracksuits & Sweats,Trading Cards,Tunic,Two-Piece,Underwear,Vases,Video Game,Video Games & Consoles,Vintage & Collectibles,Wallets,Watch,Watches,Women,Women's Accessories,Women's Handbags,Writing,Yoga & Pilates,eBook Readers,iPad
6,1477709,Victoria's Secret Alabama sleep pants,2,8.0,1,University of Alabama pajama pants. Size XS. S...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
8,711453,Black top,3,8.0,1,Black off the shoulder top. Size small,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
10,834405,Express flip flops,3,9.0,0,No description yet,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
